In [55]:
#import modules
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn.model_selection as ms
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn import tree
from sklearn import ensemble
import xgboost
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [26]:
#Import dataset
trimmed2018 = pd.read_csv('cdc2018trimmed2.csv')

In [27]:
print(trimmed2018.shape)
trimmed2018.head()

(3801534, 91)


,DOB_YY,DOB_MM,DOB_TT,DOB_WK,BFACIL3,MAGER9,RESTATUS,MRACEHISP,MAR_P,DMAR,...,CA_GAST,CA_LIMB,CA_CLEFT,CA_CLPAL,CA_DOWN,CA_DISOR,CA_HYPO,ITRAN,ILIVE,BFED
0,2018,1,1227,2,1,5,2,1,X,1.0,...,N,N,N,N,N,N,N,N,Y,Y
1,2018,1,1704,2,1,6,2,3,Y,2.0,...,N,N,N,N,N,N,N,N,Y,Y
2,2018,1,336,2,1,4,1,1,X,1.0,...,N,N,N,N,N,N,N,N,Y,Y
3,2018,1,938,2,1,3,2,3,Y,2.0,...,N,N,N,N,N,N,N,N,Y,N
4,2018,1,830,3,1,6,1,1,X,1.0,...,N,N,N,N,N,N,N,N,Y,Y


In [28]:
trimmed2018.shape

(3801534, 91)

In [29]:
trimmed2018['AB_NICU'].unique()

array(['N', 'Y', 'U'], dtype=object)

In [30]:
trimmed2018 = trimmed2018[trimmed2018['AB_NICU'].isin(['Y', 'N'])]
trimmed2018['AB_NICU'].unique()

array(['N', 'Y'], dtype=object)

In [31]:
def create_random_column(df):
    '''
    this creates a list of random numbers between 1 and 1000
    of the same lenght as each column in the dataframe, appends
    a column named "RANDOM" to the dataframe
    '''
    import random
    mylist = []
    for i in range(0,df.shape[0]):
        x = random.randint(1,1000)
        mylist.append(x)
    df['RANDOM'] = mylist
    
    return df

In [32]:
trimmed2018 = create_random_column(trimmed2018)

In [33]:
#LabelEncoding Function. Thanks Ira!
def LabelEncoding(dataframe):
    '''
    Function that takes a dataframe and transforms it with label encoding on all the categorical features.
    '''
    
    #create a list using object types since dataframe.dtypes.value_counts() only shows objects and int64
    objlist = list(dataframe.select_dtypes(include=['object']).columns)
    
    #change type then transform column using cat codes
    for col in objlist:
        dataframe[col] = dataframe[col].astype('category')
        dataframe[col] = dataframe[col].cat.codes
    
    return dataframe

In [36]:
label2018 = LabelEncoding(trimmed2018)

In [37]:
label2018['AB_NICU'].value_counts()

0    3450677
1     347404
Name: AB_NICU, dtype: int64

In [38]:
# function to split out holdout test set
def split_sets(dataframe, seed, test_prop=0.1): 
    '''
    - A function that splits specifically a dataframe into a train and test portion
    - Requires multiple assignment: train, test
    ---------------
    - dataframe: dataframe to be split
    - seed: set seed for reproducability
    - test_prop: takes a float - proportion of dataframe that should be allocated to the test set
    '''

    np.random.seed(seed)
    testIdxes = np.random.choice(range(0,dataframe.shape[0]), size=round(dataframe.shape[0]*test_prop), replace=False)
    trainIdxes = list(set(range(0,dataframe.shape[0])) - set(testIdxes))

    train = dataframe.iloc[trainIdxes,:]
    test  = dataframe.iloc[testIdxes,:]
    
    return train, test

In [39]:
# split 90:10 train, test
train2018, test2018 = split_sets(label2018, 0, test_prop=0.1)

In [44]:
#Downsample function, thanks Bettina and Aron!
def downsample_df (df):

    '''
    Remove undefined information on NICU admissions (AB_NICU == 'U'),
    create a binary target vector, and create a "balanced" dataframe
    with all NICU admissions and matching numbers of randomly selected non-NICU admissions.
    '''

    import pandas as pd
    import numpy as np

    # Get indicies of each class' observations
    index_class0 = np.where(df['AB_NICU'] == 0)[0]
    index_class1 = np.where(df['AB_NICU'] == 1)[0]

    # Get numbers of observations in class 0
    n_class1 = len(index_class1)

    # Randomly sample the same number of observations from class 1 as in class 0, without replacement
    np.random.seed(0)
    index_class0_downsampled = np.random.choice(index_class0, size=n_class1, replace=False)

    # Create dataframes for NICU and downsampled non-NICU
    df_defect = df.iloc[index_class1]
    df_adj_NONdefect = df.iloc[index_class0_downsampled]

    # Append into 1 dataframe
    df_downsampled = df_defect.append(df_adj_NONdefect)

    return df_downsampled

In [45]:
#Downsampled
dsample = downsample_df(train2018)

In [49]:
#check
print(sum(label2018.AB_NICU == 1))   #347404
print(sum(train2018.AB_NICU == 1))   #312942
print(dsample.shape)   #625884 rows, 92 columns   -> It worked! 
print(sum(dsample.AB_NICU == 0))  #312942
print(sum(dsample.AB_NICU == 1))  #312942

347404
347404
312942
(625884, 92)
312942
312942


In [52]:
X_train = dsample.drop('AB_NICU', axis=1)
y_train = dsample['AB_NICU']
X_test = test2018.drop('AB_NICU', axis=1)
y_test = test2018['AB_NICU']
y_train.value_counts()

1    312942
0    312942
Name: AB_NICU, dtype: int64

In [53]:
#XGBoost initial fit 
xgb = XGBClassifier()
xgb.set_params(random_state=0)
xgb.fit(X_train, y_train)
print("The training error is: %.5f" % (1 - xgb.score(X_train, y_train)))
print("The test error is: %.5f" % (1 - xgb.score(X_test, y_test)))

The training error is: 0.15004
The test error is: 0.10492


In [56]:
cm_test = confusion_matrix(y_test, xgb.predict(X_test))
cm_test

array([[312629,  32717],
       [  7132,  27330]])

In [57]:
#XGBoost initial fit # not needed, this is non-NICU adjusted
#xgb1 = XGBClassifier()
#xgb1.set_params(random_state=0, scale_pos_weight = 9)
#xgb1.fit(X_train, y_train)
#print("The training error is: %.5f" % (1 - xgb1.score(X_train, y_train)))
#print("The test error is: %.5f" % (1 - xgb1.score(X_test, y_test)))

The training error is: 0.43952
The test error is: 0.79275


In [58]:
#cm_test1 = confusion_matrix(y_test, xgb1.predict(X_test))
#cm_test1

array([[ 44517, 300829],
       [   263,  34199]])

In [ ]:
# set the parameter grid
xgb_param_grid ={'learning_rate': [0.01,0.05,0.1],
                 'max_depth': [3,4,5],
                 'min_child_weight': [4,5,6],
                 'n_estimators': [300, 400, 500]}

#grid search
grid_search_xgb = GridSearchCV(xgb, xgb_param_grid, scoring='balanced_accuracy', cv= 3, n_jobs=-1, return_train_score = True)
%time grid_search_xgb.fit(X_train, y_train)

In [ ]:
# get the best parameters
print(grid_search_xgb.best_params_)
print(grid_search_xgb.best_score_)

In [ ]:
# get the training/test errors
print("The training error is: %.5f" % (1 - grid_search_xgb.best_estimator_.score(X_train, y_train)))
print("The test error is: %.5f" % (1 - grid_search_xgb.best_estimator_.score(X_test, y_test)))

In [ ]:
#Confusion matrix
cm_grid = confusion_matrix(y_test, grid_search_xgb.best_estimator_.predict(X_test))
cm_grid

In [ ]:
#Prediction with tuned hyperparameters
grid_search_xgb_pred = grid_search_xgb.predict(X_test)

In [70]:
# set the parameter grid
xgb_param_grid ={'learning_rate': [0.01,0.05,0.1],
                 'max_depth': [2,3,4],
                 'min_child_weight': [3,4,5],
                 'n_estimators': [100,200,300]}

#grid search
grid_search_xgb1 = GridSearchCV(xgb, xgb_param_grid, scoring='precision', cv= 3, n_jobs=-1, return_train_score = True)
%time grid_search_xgb1.fit(X_train, y_train)

KeyboardInterrupt: 

In [ ]:
# get the best parameters
print(grid_search_xgb1.best_params_)
print(grid_search_xgb1.best_score_)

In [ ]:
# get the training/test errors
print("The training error is: %.5f" % (1 - grid_search_xgb1.best_estimator_.score(X_train, y_train)))
print("The test error is: %.5f" % (1 - grid_search_xgb1.best_estimator_.score(X_test, y_test)))

In [ ]:
cm_grid1 = confusion_matrix(y_test, grid_search_xgb1.best_estimator_.predict(X_test))
cm_grid1

In [ ]:
# set the parameter grid
xgb_param_grid ={'learning_rate': [0.01,0.05,0.1],
                 'max_depth': [4,5],
                 'min_child_weight': [3,4,5],
                 'n_estimators': [100,200,300]}

#grid search
grid_search_xgb2 = GridSearchCV(xgb, xgb_param_grid, scoring='recall', cv= 5, n_jobs=-1, return_train_score = True)
%time grid_search_xgb2.fit(X_train, y_train)

In [ ]:
# get the best parameters
print(grid_search_xgb2.best_params_)
print(grid_search_xgb2.best_score_)

In [ ]:
# get the training/test errors
print("The training error is: %.5f" % (1 - grid_search_xgb2.best_estimator_.score(X_train, y_train)))
print("The test error is: %.5f" % (1 - grid_search_xgb2.best_estimator_.score(X_test, y_test)))

In [ ]:
cm_grid2 = confusion_matrix(y_test, grid_search_xgb2.best_estimator_.predict(X_test))
cm_grid2

In [68]:
# Get numerical feature importances
importances_xgb = list(xgb.feature_importances_)

# List of tuples with variable and importance
feature_importances_xgb = [(feature, round(importance, 5)) for feature, importance in zip(X_train.columns, importances_xgb)]

# Sort the feature importances by most important first
xgb_feature_importances = sorted(feature_importances_xgb, key = lambda x: x[1], reverse = True )

# Print out the feature and importances 
[print('Variable: {:10} Importance: {}'.format(*pair)) for pair in xgb_feature_importances]

Variable: OEGest_R10 Importance: 0.2592499852180481
Variable: AB_AVEN1   Importance: 0.14889000356197357
Variable: AB_ANTI    Importance: 0.10232999920845032
Variable: APGAR5     Importance: 0.06413999944925308
Variable: ME_ROUT    Importance: 0.05477999895811081
Variable: BWTR12     Importance: 0.04682999849319458
Variable: ITRAN      Importance: 0.04029000177979469
Variable: GESTREC10  Importance: 0.03731999918818474
Variable: LD_CHOR    Importance: 0.020320000126957893
Variable: ILLB_R11   Importance: 0.013380000367760658
Variable: LD_STER    Importance: 0.013260000385344028
Variable: CA_CCHD    Importance: 0.01322999969124794
Variable: AB_AVEN6   Importance: 0.01307000033557415
Variable: RF_PDIAB   Importance: 0.013050000183284283
Variable: SEX        Importance: 0.009639999829232693
Variable: IP_HEPC    Importance: 0.009410000406205654
Variable: RF_GDIAB   Importance: 0.009069999679923058
Variable: FAGEREC11  Importance: 0.008489999920129776
Variable: PRECARE5   Importance: 0.0081

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [69]:
xgb_feature_importances_top20 = xgb_feature_importances[:20]
featureNames, featureScores = zip(*list(xgb_feature_importances_top20))
xgb_feature_importances_top20

[('OEGest_R10', 0.25925),
 ('AB_AVEN1', 0.14889),
 ('AB_ANTI', 0.10233),
 ('APGAR5', 0.06414),
 ('ME_ROUT', 0.05478),
 ('BWTR12', 0.04683),
 ('ITRAN', 0.04029),
 ('GESTREC10', 0.03732),
 ('LD_CHOR', 0.02032),
 ('ILLB_R11', 0.01338),
 ('LD_STER', 0.01326),
 ('CA_CCHD', 0.01323),
 ('AB_AVEN6', 0.01307),
 ('RF_PDIAB', 0.01305),
 ('SEX', 0.00964),
 ('IP_HEPC', 0.00941),
 ('RF_GDIAB', 0.00907),
 ('FAGEREC11', 0.00849),
 ('PRECARE5', 0.00818),
 ('LD_ANES', 0.00767)]

In [ ]:
plt.barh(range(len(featureScores)), featureScores, tick_label=featureNames)
plt.gca().invert_yaxis()
plt.title('feature importance')
plt.ylabel('Features')
plt.xlabel('Importance Score')
plt.title('Feature Importances')
plt.savefig('xgbFI.png')

In [ ]:
# Get numerical feature importances
importances_grid_search_xgb1 = list(grid_search_xgb1.feature_importances_)

# List of tuples with variable and importance
feature_importances_xgb1 = [(feature, round(importance, 5)) for feature, importance in zip(X_train.columns, importances_grid_search_xgb1)]

# Sort the feature importances by most important first
xgb1_feature_importances = sorted(feature_importances_xgb1, key = lambda x: x[1], reverse = True )

# Print out the feature and importances 
[print('Variable: {:10} Importance: {}'.format(*pair)) for pair in xgb1_feature_importances]